In [ ]:
import os
os.environ['XLA_FLAGS'] = '--xla_dump_hlo_as_text --xla_dump_to=/home/karl/tmp/hlo'
import tensorflow as tf
tf.config.optimizer.set_jit(True)

In [ ]:
from functools import reduce
from IPython.display import display
from IPython.display import Image
import contextlib
import io
import numpy as np
import PIL
import wedge
from matplotlib import pyplot as plt
from jupyter_renderer_widget import PyplotRenderer
from jupyter_renderer_widget import Renderer
from jupyter_renderer_widget import to_uint8_rgb

In [ ]:
DEFAULT_COLOR = np.array([0x8c, 0xc4, 0xfc])


def colorize(im, color=DEFAULT_COLOR):
    return tf.stack(
        [
            im * color[0],
            im * color[1],
            im * color[2],
        ],
        axis=2
    )


def display_np_image(np_image):
    rgb_image = to_uint8_rgb(np_image)
    display(PIL.Image.fromarray(rgb_image, 'RGB'))


@contextlib.contextmanager
def tf_options(options):
    old_opts = tf.config.optimizer.get_experimental_options()
    tf.config.optimizer.set_experimental_options(options)
    try:
        yield
    finally:
        tf.config.optimizer.set_experimental_options(old_opts)

In [ ]:
x_offset = 0.2599368
y_offset = 0.0016
width, height = 960, 540
initial_range = 2.
zoom_factor = 0.9


#@tf.function
#def f(z):
#    c = lambda i: i < z
#    b = lambda i: (i + 1,)
#    return tf.while_loop(c, b, [tf.constant(0)])

opts = {
    'disable_model_pruning': False,
    'disable_meta_optimizer': False,
    'arithmetic_optimization': False,
}
with tf_options(opts):
    @tf.function
    def do_render(t, niterations=100):
        x_zoom = initial_range * zoom_factor ** t
        y_zoom = x_zoom * height / width
        #print(x_zoom)
        x = tf.cast(tf.linspace(-x_zoom + x_offset, x_zoom + x_offset, width), tf.complex128)
        y = tf.cast(tf.linspace(-y_zoom + y_offset, y_zoom + y_offset, height), tf.complex128) * 1j
        x, y = tf.meshgrid(x, y)
        c = x + y
        zeros = tf.zeros([height, width])
        ones = tf.ones_like(zeros)
        
        def iterate(i, z, out):
            z = z ** 2 + c
            mask = tf.cast(z * tf.math.conj(z), tf.float32) < 2
            fill = ones * tf.cast(i / niterations, tf.float32)
            out = tf.where(mask, fill, out)
            return i + 1, z, out

        initial_state = [tf.constant(0), c, zeros]
        cond = lambda i, z, out: i < niterations
        _, _, out = tf.while_loop(cond, iterate, initial_state)
        return colorize(out)

def render_mandelbrot(t):
    return do_render(tf.Variable(float(t))).numpy()

Renderer(render_mandelbrot, frame_count=140)

In [ ]:
%timeit do_render(10)

In [ ]:
FIXED_BITS = 60


def float_to_fixed(f, bits=FIXED_BITS):
    return tf.cast(f * (1 << bits), tf.int64)


def fixed_to_float(i, bits=FIXED_BITS):
    return tf.cast(i, tf.float64) / (1 << bits)


def fixed_multiply(i1, i2, bits=FIXED_BITS):
    return tf.bitwise.right_shift(i1, bits // 2) * tf.bitwise.right_shift(i2, bits // 2)


def fixed_square(i, bits=FIXED_BITS):
    return fixed_multiply(i, i, bits)


def fixed_complex_multiply(i1, j1, i2, j2, bits=FIXED_BITS):
    return (
        fixed_multiply(i1, i2, bits) - fixed_multiply(j1, j2, bits),
        fixed_multiply(i1, j2, bits) + fixed_multiply(i2, j1, bits),
    )


def fixed_complex_square(i, j, bits=FIXED_BITS):
    return fixed_complex_multiply(i, j, i, j, bits)


def fixed_complex_abs(i, j, bits=FIXED_BITS):
    return fixed_square(i, bits) + fixed_square(j, bits)


ITERATION_COUNT = 120

x_offset = float_to_fixed(0.25993685)
y_offset = float_to_fixed(0.0016)
width, height = 960, 540
initial_range = 2.
zoom_factor = 0.9
color = np.array([0x8c, 0xc4, 0xfc])


opts = {
    'disable_model_pruning': False,
    'disable_meta_optimizer': False,
    'arithmetic_optimization': False,
}
with tf_options(opts):
    #@tf.function
    def do_render(t, niterations=100):
        xi_zoom = initial_range * zoom_factor ** t
        yi_zoom = xi_zoom * height / width
        x_zoom = float_to_fixed(xi_zoom)
        y_zoom = float_to_fixed(yi_zoom)
        threshold = float_to_fixed(2.)
        x_min, x_max = -x_zoom + x_offset, x_zoom + x_offset
        y_min, y_max = -y_zoom + y_offset, y_zoom + y_offset
        delta = (x_max - x_min) // width
        x_max = x_min + delta * width
        y_max = y_min + delta * height
        xs = tf.range(x_min, x_max, delta)
        ys = tf.range(y_min, y_max, delta)
        x0, y0 = tf.meshgrid(xs, ys)
        x, y = x0, y0
        zeros = tf.zeros([height, width])
        ones = tf.ones_like(zeros)
        
        def iterate(i, x, y, out):
            x, y = fixed_complex_square(x, y)
            x += x0
            y += y0
            mask1 = fixed_complex_abs(x, y) >= threshold
            mask2 = out == 0.
            mask = tf.math.logical_and(mask1, mask2)
            fill = ones * tf.cast(i / ITERATION_COUNT, tf.float32)
            out = tf.where(mask, fill, out)
            return i + 1, x, y, out

        initial_state = [tf.constant(0), x, y, zeros]
        cond = lambda i, x, y, out: i < niterations
        _, _, _, out = tf.while_loop(cond, iterate, initial_state)
        out = tf.where(out == 0., ones, out)
        return colorize(out)

def render_mandelbrot(t):
    return do_render(tf.Variable(float(t))).numpy()

Renderer(render_mandelbrot, frame_count=200)

In [ ]:
initial_range = 2.
zoom_factor = 0.9
color = np.array([0x8c, 0xc4, 0xfc])


opts = {
    'disable_model_pruning': False,
    'disable_meta_optimizer': False,
    'arithmetic_optimization': False,
}
with tf_options(opts):
    @tf.function
    def do_render(
        t,
        x_offset,
        y_offset,
        width,
        height,
        niterations=tf.constant(100),
        nparts=4,
    ):
        x_zoom = initial_range * zoom_factor ** t
        y_zoom = x_zoom * height / width
        xs = tf.linspace(-1., 1., width)
        ys = tf.linspace(-1., 1., height)
        x0, y0 = tf.meshgrid(xs, ys)
        x_zoom = wedge.from_float(x_zoom, nparts)
        y_zoom = wedge.from_float(y_zoom, nparts)
        x0 = wedge.from_float(x0, nparts)
        y0 = wedge.from_float(y0, nparts)
        x0 = wedge.add(wedge.multiply(x0, x_zoom), x_offset)
        y0 = wedge.add(wedge.multiply(y0, y_zoom), y_offset)
        x, y = x0, y0
        zeros = tf.zeros([height, width])
        ones = tf.ones_like(zeros)
        
        def iterate(i, x, y, out):
            x, y = wedge.complex_square(x, y)
            x = wedge.add(x, x0)
            y = wedge.add(y, y0)
            abs = wedge.complex_abs(x, y)[-1]
            mask1 = abs >= 2
            mask2 = out == 0.
            mask = tf.math.logical_and(mask1, mask2)
            fill = ones * tf.cast(i / niterations, tf.float32)
            out = tf.where(mask, fill, out)
            return i + 1, x, y, out
            
        initial_state = [tf.constant(0), x, y, zeros]
        cond = lambda i, x, y, out: i < niterations
        _, _, _, out = tf.while_loop(cond, iterate, initial_state, back_prop=False)
        out = tf.where(out == 0., ones, out)
        return colorize(out)

In [ ]:
NPARTS = 8
NITERATIONS = 600
WIDTH, HEIGHT = 96*20, 54*20
DEFAULT_X_OFFSET_F = 0.2599368542
DEFAULT_Y_OFFSET_F = 0.00159997545
DEFAULT_X_OFFSET = wedge.from_float(tf.convert_to_tensor(DEFAULT_X_OFFSET_F, tf.float32), NPARTS)
DEFAULT_Y_OFFSET = wedge.from_float(tf.convert_to_tensor(DEFAULT_Y_OFFSET_F, tf.float32), NPARTS)
DEFAULT_X_OFFSET, DEFAULT_Y_OFFSET

In [ ]:
DEFAULT_X_OFFSET = [2545, 105, 2045, 369, 256, 718, 532, 0]
DEFAULT_Y_OFFSET = [1010, 2047, 524, 0, 1604, 566, 3, 0]

def render_mandelbrot(t):
    t_var = tf.constant(float(t))
    x_offset = tf.constant(DEFAULT_X_OFFSET, wedge.DEFAULT_DTYPE)
    y_offset = tf.constant(DEFAULT_Y_OFFSET, wedge.DEFAULT_DTYPE)
    niterations = tf.constant(NITERATIONS)
    return do_render(t_var, x_offset, y_offset, WIDTH, HEIGHT, niterations, NPARTS).numpy()


Renderer(render_mandelbrot, frame_count=500, preview_frame=200)